# Superposition Kata Workbook, Part 2

The [Superposition Kata Workbook, Part 1](./Workbook_Superposition.ipynb) includes includes kata tasks 1 - 7). 

**What is this workbook?**
A workbook is a collection of problems, accompanied by solutions to them. 
The explanations focus on the logical steps required to solve a problem; they illustrate the concepts that need to be applied to come up with a solution to the problem, explaining the mathematical steps required. 

Note that a workbook should not be the primary source of knowledge on the subject matter; it assumes that you've already read a tutorial or a textbook and that you are now seeking to improve your problem-solving skills. You should attempt solving the tasks of the respective kata first, and turn to the workbook only if stuck. While a textbook emphasizes knowledge acquisition, a workbook emphasizes skill acquisition.

This workbook describes the solutions to the problems offered in the [Superposition kata](./Superposition.ipynb). 
Since the tasks are offered as programming problems, the explanations also cover some elements of Q# that might be non-obvious for a first-time user.

**What you should know for this workbook**

You should be familiar with the following concepts before tackling the Superposition kata (and this workbook):
1. Basic linear algebra
2. The concept of qubit and multi-qubit systems
3. Single-qubit and multi-qubit quantum gates

To begin, first prepare this notebook for execution (if you skip this step, you'll get "Syntax does not match any known patterns" error when you try to execute Q# code in the next cells):

In [ ]:
%package Microsoft.Quantum.Katas::0.9.1909.3002

> The package versions in the output of the cell above should always match. If you are running the Notebooks locally and the versions do not match, please install the IQ# version that matches the version of the `Microsoft.Quantum.Katas` package.
> <details>
> <summary><u>How to install the right IQ# version</u></summary>
> For example, if the version of `Microsoft.Quantum.Katas` package above is 0.1.2.3, the installation steps are as follows:
>
> 1. Stop the kernel.
> 2. Uninstall the existing version of IQ#:
>        dotnet tool uninstall microsoft.quantum.iqsharp -g
> 3. Install the matching version:
>        dotnet tool install microsoft.quantum.iqsharp -g --version 0.1.2.3
> 4. Reinstall the kernel:
>        dotnet iqsharp install
> 5. Restart the Notebook.
> </details>


## <a name="superposition-of-all-even-or-all-odd-numbers"></a>Task 10. Superposition of all even or all odd numbers.

**Inputs:** 

1. $N$ ($N \ge 1$) qubits in the $|0 \dots 0\rangle$ state (stored in an array of length $N$).
2. A boolean `isEven`.

**Goal:**  Prepare a superposition of all *even* numbers if `isEven` is `true`, or of all *odd* numbers if `isEven` is `false`.  
A basis state encodes an integer number using [big-endian](https://en.wikipedia.org/wiki/Endianness) binary notation: state $|01\rangle$ corresponds to the integer $1$, and state $|10 \rangle$ - to the integer $2$.  

> For example, for $N = 2$ and `isEven = false` you need to prepare superposition $\frac{1}{\sqrt{2}} \big (|01\rangle + |11\rangle\big )$,  
and for $N = 2$ and `isEven = true` - superposition $\frac{1}{\sqrt{2}} \big (|00\rangle + |10\rangle\big )$.
</details>

### Solution

Let’s look at some qubit examples to illustrate the binary numbering system. For N = 2, the qubits can be in the following states (ignoring the $\frac{1}{\sqrt{2}}$ for now), where the left column represents the desired states for isEven = false and the right column represents the desired states for isEven = true:

<img src="./img/Task10_1.png" width="400">
 
 If we apply the binary numbering system to three qubits the pattern becomes more obvious:
 
<img src="./img/Task10_2.png" width="400">
 
 The state of the two left most qubits is the same for isEven = false and for isEven = true. A quick sanity check for four qubits re-confirms the pattern:
 
<img src="./img/Task10_3.png" width="400">
 
Again, the left most qubits are the same for both sets, and it is only the right most qubit that is different. 
The solution is to put all but the right most qubit into an equal superposition (as in Task 9) and to set the right most qubit to $|0\rangle$ or $|1\rangle$ depending on the isEven flag. For the last qubit we will use the NOT operator to convert $|0\rangle$ to $|1\rangle$ if isEven = false.

In [ ]:
%kata T10_EvenOddNumbersSuperposition_Test

operation EvenOddNumbersSuperposition (qs : Qubit[], isEven : Bool) : Unit is Adj {
    mutable N = Length(qs);
    for (i in 0 .. N-2) {
        H(qs[i]);
    }
    // for odd numbers, flip the last bit to 1
    if (not isEven) {
        X(qs[N-1]);
    }
}

[Return to task 10 of the Superposition kata.](./Superposition.ipynb#superposition-of-all-even-or-all-odd-numbers)

## <a name="threestates-twoqubits"></a> Task 11*. $\frac{1}{\sqrt{3}} \big(|00\rangle + |01\rangle + |10\rangle\big)$ state.

**Input:** Two qubits in the $|00\rangle$ state.

**Goal:**  Change the state of the qubits to $\frac{1}{\sqrt{3}} \big(|00\rangle + |01\rangle + |10\rangle\big)$.

<br/>
<details>
  <summary>Need a hint? Click here</summary>
  If you need trigonometric functions, you can find them in Microsoft.Quantum.Math namespace; you'll need to add <pre>open Microsoft.Quantum.Math;</pre> to the code before the operation definition.
</details>

### Conventional solution

Looking at the goal state we notice two points of interest:
1. The goal state is missing |11$\rangle$ state.
2. The normalization factor is $\frac{1}{\sqrt{3}}$.

Let’s discuss the first observation. Assume our solution places each qubit into a superposition state as follows: 

\begin{align*}
    |q_1\rangle = a|0\rangle + b|1\rangle \\    
    |q_2\rangle = c|0\rangle + d|1\rangle
\end{align*}
Then the combined state would be:
 
\begin{align*}
   |q_1q_2\rangle= &\big(a|0\rangle + b|1\rangle\big) \otimes \big(c|0\rangle + d|1\rangle\big)\\
                 = &ac|00\rangle + ad|01\rangle + bc|10\rangle + bd|11\rangle\\                     
\end{align*}

To reach the goal state we would need the |11$\rangle$ term to disappear. This means we would need either b = 0 or d = 0. However, if b = 0 then the |10$\rangle$ term would also disappear. And if d = 0 then the |01$\rangle$ term would also disappear. Therefore, our first solution will start off with placing only one qubit in the superposition state (the alternative solution does create all four states, but we will discuss this later).

The second observation suggests that we are not going to be using a Hadamard operation to place a qubit into a superposition state, which causes the normalization factor to be $\frac{1}{\sqrt{2}}$ instead of $\frac{1}{\sqrt{3}}$.

Let’s us assume that we have some yet to be specified rotation operation R1:

<table style="background-color: white; border:0 solid; tr  { background-color:white; }">
    <col width=200>
    <col width=200>
        <td style="text-align:center; background-color:white; border:0"><img src="./img/task11R1RotationOperation.png"/></td>
        <td style="text-align:center; background-color:white; border:0">$R1|0\rangle = \sqrt\frac{1}{3} |0\rangle + \sqrt\frac{2}{3} |1\rangle$</td>       
</table>

Applying the R1 rotation to the left qubit we get a new state with two superposition terms: 

$$R1(|0\rangle) \otimes |0\rangle =  \sqrt\frac{1}{3}|00\rangle + \sqrt\frac{2}{3}|10\rangle = |\psi\rangle \:\:\:  (Equation 1)$$

We now need to get the third term. A discussed above, placing the second qubit (right qubit) into superposition will gives us all four terms, which is not what we want. Instead, we want to place one of the two terms into an additional superposition state, i.e. apply an operation that places only |00$\rangle$  or |10$\rangle$ into a new superposition. Because an operation on a qubit will act on all of the qubit’s superpositions states, we will need an operation that will use some discriminating criteria or condition to perform a no-op when encountering one superposition term, and create a new superposition state when encountering the other superposition term. Looking at the terms, we notice that they vary in the left qubit. 

$$|\psi\rangle = \sqrt\frac{1}{3}|{\color{Red} 0}0\rangle + \sqrt\frac{2}{3}|{\color{Red} 1}0\rangle   \:\:\:  (Equation 1)$$

The state of the left qubit can be used as the control bit for a controlled operation. If the left qubit is in |0$\rangle$ then do nothing otherwise if it is in |1$\rangle$ then apply an operation. The operation that is applied will create a new superposition state to give us three terms. 

Let’s assume the new operation is another rotation that we will call R2, where the c and d coefficients are still to be determined:

$$R2|0\rangle = c|0\rangle + d|1\rangle$$

Applying R2 to the right qubit on |$\psi\rangle$:

$$\underset{\text{Left qubit = Control qubit} \\ \text{Right qubit = Target qubit}}{\underbrace{Controlled-R2}}|\psi\rangle= \sqrt\frac{1}{3}|00\rangle + \sqrt\frac{2}{3} |1\rangle \otimes (c|0\rangle + d|1\rangle)= \sqrt\frac{1}{3}|00\rangle + \sqrt\frac{2}{3} c|10\rangle + \sqrt\frac{2}{3}d|11\rangle = |\Phi\rangle$$

We now have the three terms, but they are not yet in the target state.

<table align="center" style="background-color: white; border:1px solid; tr  { background-color:transparent; }">
    <col width=300>
    <col width=300>
    <tr>
        <th style="text-align:center; border:1px solid">Goal State</th>
        <th style="text-align:center; border:1px solid">Current State</th>
    </tr>
    <tr>
        <td style="text-align:center; border:1px solid">|00$\rangle$, |01$\rangle$, |10$\rangle$</td>
        <td style="text-align:center; border:1px solid">|00$\rangle$, |10$\rangle$, |11$\rangle$</td>   
      </tr>
</table> 

We can now apply a CNOT (Controlled NOT) operation using the right qubit as the control bit and the left qubit as the target bit:

<table align="center" style="background-color: white; border:0px solid; tr  { background-color:transparent; }">
    <col width=300>
    <col width=300>
    <tr>
        <th style="text-align:center; border:1px solid">Current State</th>
        <th style="text-align:center; border:1px solid">After CNOT Operation</th>
    </tr>
    <tr>
        <td style="text-align:center; border:1px solid">|00$\rangle$ </td>
        <td style="text-align:center; border:1px solid">|00$\rangle$</td>   
      </tr>     
    <tr>
        <td style="text-align:center; vertical-align:top; border:1px solid">|10$\rangle$</td>
        <td style="text-align:center; border:1px solid">|10$\rangle$</td>   
      </tr>
      <tr>
        <td style="text-align:center; vertical-align:top; border:1px solid">|11$\rangle$</td>
        <td style="text-align:center; border:1px solid">|01$\rangle$</td>   
      </tr>
</table>

Repeating the last equation here for convenience, we have:
 
$$|\Phi\rangle = \sqrt\frac{1}{3} |00\rangle + \sqrt\frac{2}{3}  \cdot  c|10\rangle  + \sqrt\frac{2}{3} \cdot d|11\rangle$$

And after applying the CNOT operation we get:

$$\underset{\text{Right qubit = Control qubit} \\ \text{Left qubit = Target qubit}}{\underbrace{CNOT}}|\Phi\rangle = \sqrt\frac{1}{3} |00\rangle + \sqrt\frac{2}{3} \cdot c |10\rangle + \sqrt\frac{2}{3} \cdot d |01\rangle$$

We now have the 3 desired superposition states.

Before we resolve the unknow angle alpha for the first rotation, and the unknown coefficients c and d from the second rotation, let’s step back and revisit equation 1:

$$|\psi\rangle = \sqrt\frac{1}{3}|{\color{Red} 0}0\rangle + \sqrt\frac{2}{3}|{\color{Red} 1}0\rangle   \:\:\:  (Equation 1)$$

In the discussion above we decided to use a Controlled operation that acts if the control bit is in the |1$\rangle$ state. However, Q# easily allows us to perform a Controlled operation if the control bit is in the |0$\rangle$ state (and thus no operation if the control bit is in the |1$\rangle$ state), using the function <a href="https://docs.microsoft.com/en-us/qsharp/api/qsharp/microsoft.quantum.canon.controlledonint?view=qsharp-preview">ControlledOnInt:</a> Microsoft.Quantum.Canon.ControlledOnInt function. Quoting the link, it says the function “Returns a unitary operator that applies an oracle on the target register if the control register state corresponds to a specified positive integer.”

If the left qubit is in |0$\rangle$ state then apply Rotation R2, otherwise if it is in the |1$\rangle$ state then do nothing. Starting from equation 1, again, we apply R2 to the right qubit on |$\psi\rangle$: 


$$\underset{\text{Left qubit = Control qbit} \\ \text{Right qubit = Target qubit}}{\underbrace{Controlled-R2}}|\psi\rangle= \sqrt\frac{1}{3}|0\rangle \otimes (c|0\rangle + d|1\rangle) + \sqrt\frac{2}{3} |0\rangle = \sqrt\frac{1}{3}\cdot c|00\rangle + \sqrt\frac{1}{3} \cdot d|01\rangle + \sqrt\frac{2}{3}|10\rangle $$

By reverting the state of the control bit on which the Controlled operation gets involved we immediately get the desired superposition states and avoid calling the CNOT operation. However, our coefficient for the |10> term is incorrect; it is $\sqrt\frac{2}{3}$ instead of $\sqrt\frac{1}{3}$. We can fix this by changing the first rotation from: $R1|0\rangle = \sqrt\frac{1}{3} |0\rangle + \sqrt\frac{2}{3} |1\rangle$ , to: $R1|0\rangle = \sqrt\frac{2}{3} |0\rangle + \sqrt\frac{1}{3} |1\rangle$
<table align="center" style="background-color: white; border:0 solid; tr  { background-color:white; }">
    <col width=300>  
    <tr>
    <td style="text-align:center; vertical-align:top; background-color:white; border:0"><img align="center" src="./img/task11R1RotationOperation2.png"/></td>  
</table>

For clarity, lets repeat all steps for the optimized solution.

We define our first rotation R1 as:

$$R1|0\rangle = \sqrt\frac{2}{3} |0\rangle + \sqrt\frac{1}{3} |1\rangle$$

We apply the rotation R1 to the left qubit of the initial (problem) state:

$$R1(|0\rangle)\otimes|0\rangle = \sqrt\frac{2}{3} |00\rangle + \sqrt\frac{1}{3} |10\rangle = |\psi\rangle \:\:\:  (Equation 1(b))$$

We define our second rotation R2 as:

$$R2|0\rangle = c|0\rangle + d|1\rangle$$

We use the Q# ControlledOnInt function to apply the rotation R2 on the right qubit when the left qubit is in the $|0\rangle$ state:

$$\underset{\text{Left qubit = Control qubit} \\ \text{Right qubit = Target qubit} \\ \text{Perform operation when Control qubit = |0>}}{\underbrace{Controlled-R2}}|\psi\rangle = \sqrt\frac{2}{3}|0\rangle \otimes (c|0\rangle + d|1\rangle) + \sqrt\frac{1}{3} |10\rangle = \sqrt\frac{2}{3}c\cdot|00\rangle + \sqrt\frac{2}{3} d\cdot|01\rangle + \sqrt\frac{1}{3}d|10\rangle = |\Phi\rangle$$

Now, we have to find a value for the c coefficient and the d coefficient. The desired normalization factor is $\sqrt\frac{1}{3}$, thus if c == d == $\sqrt\frac{1}{2}$ we get:

$$|\Phi\rangle=\sqrt\frac{2}{3}c\cdot|00\rangle + \sqrt\frac{2}{3} d\cdot|01\rangle + \sqrt\frac{1}{3}|10\rangle=\sqrt\frac{2}{3}\sqrt\frac{1}{2}\cdot|00\rangle + \sqrt\frac{2}{3} \sqrt\frac{1}{2}\cdot|01\rangle + \sqrt\frac{1}{3}|10\rangle=$$
$$=\sqrt\frac{1}{3}|00\rangle + \sqrt\frac{1}{3} |01\rangle + \sqrt\frac{1}{3}|10\rangle$$

This is the desired ed state!

We can now fill out the missing coefficients for rotation R2:

$$R2|0\rangle = c|0\rangle + d|1\rangle$$

Because c == d == $\sqrt\frac{1}{2}$ rotation R2 becomes:

$$R2|0\rangle = \sqrt\frac{1}{2}|0\rangle + \sqrt\frac{1}{2}|1\rangle$$

We recognize that this is the Hadamard operation!
 
To obtain the angle alpha for the first rotation we simple calculate the arcsin of $\sqrt\frac{1}{3}$. The Q# operation for Ry expects the angle to be doubled, so the first rotation is then achieved using the following Q# statements:

$\texttt{let theta = ArcSin(1.0 / Sqrt(3.0));}$

$\texttt{Ry(2.0 * theta, qs[0]);}$


Below is the entire Q# solution. Note that it also contains a reference to another explanation.


In [ ]:
%kata T11_ThreeStates_TwoQubits_Test

operation ThreeStates_TwoQubits (qs : Qubit[]) : Unit is Adj {

    // Follow Niel's answer at https://quantumcomputing.stackexchange.com/a/2313/

    // Rotate first qubit to (sqrt(2) |0⟩ + |1⟩) / sqrt(3) (task 1.4 from BasicGates kata)
    let theta = ArcSin(1.0 / Sqrt(3.0));
    Ry(2.0 * theta, qs[0]);

    // Split the state sqrt(2) |0⟩ ⊗ |0⟩ into |00⟩ + |01⟩
    (ControlledOnInt(0, H))([qs[0]], qs[1]);
}


### Alternative solution

The alternative solution addresses the problem of eliminating the |11> term. This solution requires one to understand measurements. Let’s do a brief recap of measurement.

Assume we have a qubit in the following superposition state:

$$|\psi\rangle = a \cdot|0\rangle + b \cdot|1\rangle$$

Measuring this qubit will force the superposition state into the $|0\rangle$ or into the $|1\rangle$ state. In essence, the superposition state is being collapsed into one of the orthogonal states, and information about the original state it lost.

<img align="center" width=800 src="./img/task11Measure.png"/>

This “destructive” nature of measurement can be used to reduce the number of superposition states. We will start by applying the Hadamard operation to both qubits:

$$H(|0\rangle)\otimes H(|0\rangle)= \frac{1}{2}\big(|00\rangle + |01\rangle+ |10\rangle+ |11\rangle\big)=|\psi\rangle $$

We now need to eliminate the $|11\rangle$ term. If we were to take a measurement on any of the two qubits we would lose two terms. For example, if a measurement of the left qubit would have resulted in $|0\rangle$ it would have meant we would have lost the $|10\rangle + |11\rangle$ terms. On the other hand, if the same measurement would have resulted in $|1\rangle$ it would have meant we would have lost the $|00\rangle + |01\rangle$ terms. Therefore, we will need to use an additional qubit that we can “lose” (collapse away) after measurement. Let’s introduce a third (ancillary) qubit to $|\psi\rangle$:

 $$|\psi\rangle \otimes |0\rangle=\frac{1}{2}\big(|00\rangle + |01\rangle+ |10\rangle+ |11\rangle\big)\otimes |0\rangle=\frac{1}{2}\big(|000\rangle + |010\rangle+ |100\rangle+ |110\rangle\big) $$
 
A measurement will cause the superposition state to collapse to one of the bases that are being used in the measurement. As it is our intention to collapse away the $|11\rangle$ superposition state, we need to move it onto a different basis than the first three superposition states.

<img align="center" width=600 src="./img/task11Superposition1.png"/>

The question is then how can we convert the right most superposition state from $|110\rangle$ to $|111\rangle$? 

Looking at the qubits state we can see that it’s distinguishing feature is that the left and middle qubit are in the $|1\rangle$ state. This suggests that we can use a Controlled NOT operation.
Looking at the Microsoft documentation for <a href="https://docs.microsoft.com/en-us/quantum/language/type-model?view=qsharp-preview#controlled">"Controlled"</a> operations we have:
“In Q#, controlled versions always take an array of control qubits, and the specified state is always for all of the control qubits to be in the computational (PauliZ) One state, $|1\rangle$.” 

In our case, the array of control qubits will contain the left and middle qubit, and the rightmost qubit (our ancillary qubit) will be the target bit. 

Now, all control qubits have to be in $|1\rangle$ for the operation to happen on the target qubit. For a state in superposition, this means the superposition expression needs to be split into the individual terms to see whether the operation will be invoked.

<img align="center" width=500 src="./img/task11Superposition2.png"/>

Performing a Controlled NOT (Controlled X) using the two leftmost qubits as the control bits and the rightmost qubit as the target convert the superposition state into:

$$|\phi\rangle =\frac{1}{2}\big(|000\rangle + |010\rangle+ |100\rangle+ |111\rangle\big)=\frac{1}{2}\big(|00\rangle + |01\rangle + |10\rangle\big)\otimes |0\rangle + |11\rangle \otimes |1\rangle$$

If we now perform a measurement of the rightmost (ancillary) qubit, we will get one of the two results:

<table align="center" style="background-color: white; border:0px solid; tr  { background-color:transparent; }">
    <col width=300>
    <col width=300>
    <tr>
        <th style="text-align:center; border:1px solid">Result of measuring ancillary qubit</th>
        <th style="text-align:center; border:1px solid">Collapsed superposition state</th>
    </tr>
    <tr>
        <td style="text-align:center; border:1px solid">|0$\rangle$ </td>
        <td style="text-align:center; border:1px solid">$\frac{1}{\sqrt{3}}\big(|00\rangle + |01\rangle + |10\rangle\big)$</td>   
      </tr>     
    <tr>
        <td style="text-align:center; vertical-align:top; border:1px solid">|1$\rangle$</td>
        <td style="text-align:center; border:1px solid">|11$\rangle$</td>   
      </tr>
</table>

If we measure the ancillary qubit to be $|0\rangle$ we are done. If we measure the ancillary qubit to be $|1\rangle$ we will need to repeat the entire process. We keep repeating the process until we measure a $|0\rangle$.

In terms of measurement, we want to use the $|0\rangle$ or the $|1\rangle$ basis, i.e.

$|0\rangle=\begin{pmatrix} 1 \\ 0 \end{pmatrix}$, $|1\rangle=\begin{pmatrix} 0 \\ 1 \end{pmatrix}$ and can use the Pauli Z basis: $Z=\begin{pmatrix}1 & 0\\0 & -1\end{pmatrix}$

Q# defines the <a href="https://docs.microsoft.com/en-us/qsharp/api/qsharp/microsoft.quantum.measurement.mresetz?view=qsharp-preview">"MResetZ"</a> operation: “Measures a single qubit in the Z basis, and resets it to the standard basis state |0〉 following the measurement.”
 
Q# also defines a language construct to keep a process repeating until a desired state is reached, the construct being called the <a href="https://docs.microsoft.com/en-us/quantum/language/statements?view=qsharp-preview#repeat-until-success-loop">“Repeat-Until-Success Loop”</a>: “The loop body is executed, and then the condition is evaluated. If the condition is true, then the statement is completed; otherwise, the fixup is executed, and the statement is re-executed starting with the loop body.”

<img align="center" width=400 src="./img/Task11RepeatUntilSuccessLoop.png"/>

Therefore, the complete Q# solution is:

In [ ]:
%kata T11_ThreeStates_TwoQubits_Test

operation ThreeStates_TwoQubits (qs : Qubit[]) : Unit {
    using (ancilla = Qubit()) {
        repeat {
            // Create |00⟩ + |01⟩ + |10⟩ + |11⟩ state
            ApplyToEach(H, qs);
            // Create (|00⟩ + |01⟩ + |10⟩) ⊗ |0⟩ + |11⟩ ⊗ |1⟩
            Controlled X(qs, ancilla);
            let res = MResetZ(ancilla);
        }
        until (res == Zero)
        fixup {
            ResetAll(qs);
        }
    }
}

[Return to task 11 of the Superposition kata.](./Superposition.ipynb#threestates-twoqubits)

*Solutions to the rest of the tasks coming up soon...*